# OOI Seafloor Camera Open Datasets Example
This Jupyter notebook shows how to load video data from the [OOI Seafloor Camera](https://oceanobservatories.org/instrument-class/camhd/) system deployed at Axial Volcano on the Juan de Fuca Ridge. It demonstrates the basic usage of the pycamhd library, which can be used to extract frames from the ProRes encoded Quicktime files. These data are hosted on Microsoft's [Azure Open Datasets](https://azure.microsoft.com/en-us/services/open-datasets/).

## Import some required packages
All of these should be available through Conda on the conda-forge channel.

In [1]:
import numpy as np
import pycamhd as camhd
import pandas as pd
import fsspec
import matplotlib.pyplot as plt
from ipywidgets import interact
from ipywidgets import IntSlider

## Load the JSON file that lists all available video files
This file has useful information about each file in Azure Open Datasets, such as the Unix timestamp (seconds) of the first frame in each video, and the total number of frames in each video.

In [2]:
dbcamhd_url = 'https://ooiopendata.blob.core.windows.net/camhd/dbcamhd.json'
with fsspec.open(dbcamhd_url) as f:
    dbcamhd = pd.read_json(f, orient="records", lines=True)
dbcamhd.tail()

,name,url,filesize,md5,moov,timestamp,deployment,frame_count
16935,CAMHDA301-20200906T001500.mov,https://ooiopendata.blob.core.windows.net/camh...,15442329475,d25486f5d6f0f1611c8a6577f1fa9435,True,1599350410,7,26708
16936,CAMHDA301-20200906T031500.mov,https://ooiopendata.blob.core.windows.net/camh...,15449260579,7a3c09abda5bcdd9aef3dc186450b8e2,True,1599361211,7,26738
16937,CAMHDA301-20200906T061500.mov,https://ooiopendata.blob.core.windows.net/camh...,15552082455,b481471f03eb5b1f1500751ff30e9397,True,1599372010,7,26709
16938,CAMHDA301-20200906T091500.mov,https://ooiopendata.blob.core.windows.net/camh...,15482894883,5ae6cfbc09ebca9a6788b8a2d830ca92,True,1599382810,7,26738
16939,CAMHDA301-20200906T121500.mov,https://ooiopendata.blob.core.windows.net/camh...,15512083839,9fbdc5ec2655aa3b035a9b6b4a387754,True,1599393610,7,26723


## Choose a file to work with
Here we choose a file from "Deployment 4" which occurred in 2017.

In [3]:
mov = dbcamhd.iloc[7000]
mov

name                               CAMHDA301-20170920T181500.mov
url            https://ooiopendata.blob.core.windows.net/camh...
filesize                                             15250813073
md5                             eaadcc47dcaf8c7a94bb5b9708179db8
moov                                                        True
timestamp                                             1505930402
deployment                                                     4
frame_count                                                26953
Name: 7000, dtype: object

## Show a single video frame
We include a slider which allows any frame from this video to be shown.

In [4]:
def show_image(frame_number):
    plt.rc('figure', figsize=(12, 6))
    plt.rcParams.update({'font.size': 8})
    frame = camhd.get_frame(mov.url, frame_number)
    fig, ax = plt.subplots();
    im1 = ax.imshow(frame);
    plt.yticks(np.arange(0,1081,270))
    plt.xticks(np.arange(0,1921,480))
    plt.title('Deployment: %s    File: %s    Frame: %s' % (mov.deployment, mov['name'], frame_number));

In [6]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
initial_frame = 8060
frame_slider = IntSlider(min=0, max=mov.frame_count-1, step=1, value=initial_frame, continuous_update=False)
interact(show_image, frame_number=frame_slider);

interactive(children=(IntSlider(value=8060, continuous_update=False, description='frame_number', max=26952), O…